# **Policy to Admin Model**
AI5기 정세연, 도현진

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

from khaiii import KhaiiiApi

In [4]:
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'AppleGothic'

pd.set_option('display.max_colwidth', None)

- **최신 데이터 업로드**

In [8]:
df = pd.read_csv('policy_final_01182022.csv')

- **카테고리 별 특성공학 & 모델링**

## 기업규모

In [ ]:
# 기업 규모 분류 함수

def co_size(context):
    temp = []
    if '중소' in context:
        temp.append('중소기업')
    elif '소기업' in context:
        temp.append('소기업')
    if '공기업' in context:
        temp.append('공기업')
    if '중견' in context:
        temp.append('중견기업')
    if '대기업' in context:
        temp.append('대기업')
    if '소상공인' in context or '생산자' in context:
        temp.append('소상공인')
        
    if len(temp) == 0 or len(temp)==5:
        return '무관'
    else:
        return ', '.join(temp)



## 주요분야

### 대분류 
- A (농임어업), B (광업), C, ...

In [9]:
# 주요 분야 분류 함수


# 필요한 콜럼들 합치기
df['context'] = df['서비스명'].astype(str) + df['소관기관'].astype(str)\
    + df['소관기관유형'].astype(str) + df['선정기준'].astype(str)\
    + df['서비스목적'].astype(str) + df['지원대상'].astype(str)


api = KhaiiiApi()

# stopwords 제거
common_words = ['지원', '자격', '요건', '대상', '프로그램', '의사', \
                '경우', '연도', '사용', '개선', '시설', '행위', '선정'\
               '대상자']

# 필요한 명사 khaiii 사용해 추출
def extract_nouns(context):
    analyzed = api.analyze(context)
    
    token_list = []
    
    for i in analyzed:
        for morph in i.morphs:
            if morph.tag == 'NNP' or morph.tag == 'NNG':
                if morph.lex not in common_words and len(morph.lex)>1:
                    token_list.append(morph.lex)
    return list(set(token_list))


df['context_tokens'] = df['context'].apply(extract_nouns)


# 분류 함수

def big_category(context):
    
    categories = []
    
    if '해양' in context or '어업' in context or '농업' in context \
        or '농가' in context or '축산' in context or '농축' in context\
        or '농식품' in context or '축산물' in context or '임업' in context\
        or '천일염' in context or '염전' in context or '염업' in context:
        categories.append('농임어업')
      
    if '석탄' in context or '원유' in context or '천연가스' in context \
        or '채굴' in context or '광업' in context:
        categories.append('광업')
    
    if '제조' in context or '제조업' in context or '식료품' in context \
        or '담배' in context or '섬유제품' in context or '가방' in context\
        or '신발' in context or '의복' in context or '화학제품' in context\
        or '의약품' in context or '부품' in context or '전기장비' in context\
        or '기계' in context or '장비' in context or '유리' in context:
        categories.append('제조업')
    
    if '전기' in context or '가스' in context or '증기' in context \
        or '공기' in context or '전기업' in context or '연료용' in context\
        or '배관' in context or '공금업' in context:
        categories.append('전가증공급업')
    
    if '수도' in context or '하수' in context or '폐기물' in context \
        or '처리' in context or '수도업' in context or '폐수' in context\
        or '분뇨' in context or '재생업' in context or '정화' in context\
        or '복원업' in context:
        categories.append('수하폐원재생업')
    
    if '건설업' in context or '건설' in context or '토목' in context \
        or '기반조성' in context or '시설물' in context or '축조' in context\
        or '건물설비' in context or '설치' in context or '공사업' in context\
        or '유지관리' in context or '건설장비' in context or '운영업' in context\
        or '건축' in context or '공사' in context:
        categories.append('건설업')
    
    if '도매' in context or '소매' in context or '자동차' in context \
        or '부품' in context or '중개업' in context or '판매업' in context\
        or '소매업' in context:
        categories.append('도소매업')
    
    if '운수' in context or '창고업' in context or '운송' in context \
        or '운송업' in context or '창고' in context or '화물' in context\
        or '여객' in context or '소화물' in context or '항공' in context\
        or '취급업' in context or '운송관련' in context or '운영업' in context\
        or '실내건축' in context or '건축마무리' in context:
        categories.append('운수창고업')
    
    if '숙박' in context or '숙박업' in context or '음식점' in context \
        or '음식점업' in context or '숙박시설' in context or '호텔' in context\
        or '여관' in context or '휴양' in context or '콘도' in context\
        or '민박' in context or '기숙사' in context or '고시원' in context:
        categories.append('숙박음식점업')    
        
    if '정보통신' in context or '정보통신업' in context or '출판' in context \
        or '서적' in context or '인쇄물' in context or '교과서' in context\
        or '간행물' in context or '신문' in context or '잡지' in context\
        or '소프트웨어' in context or '개발' in context or '게임' in context\
        or '블록체인' in context or '영상' in context or '비디오물' in context\
        or '배급업' in context or '제작업' in context or '방송' in context\
        or '프로그램' in context or '영화' in context or '원판' in context\
        or '오디오물' in context or '라디오' in context or '텔레비전' in context\
        or '우편' in context or '통신' in context or '통신업' in context\
        or '공영' in context or '유선' in context or '무선' in context\
        or '프로그래밍' in context or '시스템' in context or '컴퓨터시설' in context\
        or '정보서비스업' in context or '인터넷' in context or '호스팅' in context\
        or '자료처리' in context or '뉴스' in context:
        categories.append('정보통신업')  
        
    if '금융' in context or '금융업' in context or '은행' in context \
        or '중앙은행' in context or '일반은행' in context or '신용조합' in context\
        or '저축' in context or '신용' in context or '신탁' in context\
        or '신탁업' in context or '투자' in context or '투자업' in context\
        or '보험' in context or '보험업' in context or '생명보험' in context\
        or '손해' in context or '보증보험' in context or '사회보장' in context\
        or '재보험' in context or '재보험업' in context or '연금' in context\
        or '공제' in context or '공제업' in context or '금융시장' in context\
        or '증권' in context or '선물' in context or '융자' in context:
        categories.append('금융보험업')   
        
    if '부동산' in context or '부동산업' in context or '임대' in context \
        or '주거용' in context or '비주거용' in context\
        or '부동산중개' in context or '주택' in context:
        categories.append('부동산업')   
        
    if '개발' in context or '기술' in context or '연구' in context \
        or '산업' in context or '에너지사용량' in context or '엔지니어링' in context\
        or '대학' in context or '자연과학' in context or '공학' in context\
        or '사회과학' in context or '법무' in context or '회계' in context\
        or '세무' in context or '광고' in context or '광고업' in context\
        or '시장조사' in context or '여론조사' in context or '여론' in context\
        or '컨설팅' in context or '경영' in context or '설계' in context\
        or '측량' in context or '지질' in context or '수의업' in context\
        or '디자인업' in context or '사진' in context or '촬영' in context:
        categories.append('전문과학기술업')   
        
        
    business = ['사업시설', '조경', '방제', '유지관리', '알선', '여행', '여행사',\
                '여행사업', '사무', '운송장비', '재산권']
    
    for i in business:
        if i in context and '사업시설임대업' not in categories:
            categories.append('사업시설임대업')
            
            
    administration = ['행정', '국방', '사회보장', '입법', '정부기관', '사회서비스',\
                      '노동', '외무', '공공질서', '사법']
        
    for i in administration:
        if i in context and '공공행정' not in categories:
            categories.append('공공행정')
            
            
    education = ['교육기관', '초등', '중등', '고등', '특수학교', '외국인학교',\
                '대안학교', '초등학교', '중학교', '고등학교', '교습학원'\
                '예술학원', '외국어학원', '학원', '훈련기관', '훈련학원']
        
    for i in education:
        if i in context and '교육업' not in categories:
            categories.append('교육업') 
    
    
            
    medicine = ['보건', '보건업', '사회복지', '병원', '의원', '의료업',\
                '공중보건', '복지시설', '운영업', '의료', '보육시설']
        
    for i in medicine:
        if i in context and '보건사회복지업' not in categories:
            categories.append('보건사회복지업') 
    
    
    leisure = ['예술', '스포츠', '여가', '창작', '공연시설', '공연',\
                '도서관', '사적지', '박물관', '식물원', '동물원'\
              '오락', '경기장', '골프장', '스포츠시설', '유원지'\
              '낚시', '오락장', '수상오락', '사행시설', '무도장'\
              '체육공원', '복권', '낚시장', '노래방', '게임방'\
              'PC방', '경주장', '볼링장', '당구장', '연습장', '스키장']
        
    for i in leisure:
        if i in context and '여가관련업' not in categories:
            categories.append('여가관련업') 
    
    
    organization = ['협회', '단체', '노동조합', '종교', '시민운동', '수리',\
                '수리업', '미용', '미용업', '마사지', '욕탕'\
              '세탁', '세탁업', '장례식장', '화장터', '간병'\
              '장묘', '점술', '예식장', '사행시설', '무도장'\
              '체육공원', '복권', '낚시장', '노래방', '게임방'\
              'PC방', '경주장', '볼링장', '당구장', '연습장', '스키장']
        
    for i in organization:
        if i in context and '협회단체서비스업' not in categories:
            categories.append('협회단체서비스업') 
    
    
    international = ['주한', '외국기관', '국제', '외국']
        
    for i in international:
        if i in context and '국제외국기관' not in categories:
            categories.append('국제외국기관') 
            
    
    if len(categories) == 0:
        return '기타'
    else:
        return ', '.join(categories)
    

### 중분류
- 숫자 기반

In [10]:
# 분류 함수

def small_category(context):
    
    categories = []            
    
    
    if '농업' in context or '작물' in context or '재배' in context\
        or '농가' in context or '축산' in context or '농축' in context\
        or '농식품' in context or '축산물' in context:
        
        if '중개' in context or '도매' in context:
            categories.append('462') # 상품 중개업
            categories.append('4631') # 신선 식품및 단순 가공 식품도매업
        elif '소매' in context:
            categories.append('472') # 음,식료품 담배 소매업
        else:
            categories.append('01') # 농업
            
    
    if '임업' in context or '벌목' in context or '영림' in context\
        or '임산물' in context:
        if '중개' in context or '도매' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('02') # 임업
    
    if '해양' in context or '어업' in context or '원양' in context\
        or '연근해' in context or '내수면' in context or '양식' in context\
        or '수산물' in context:
        categories.append('03') # 어업
      
    if '석탄' in context or '원유' in context or '천연가스' in context \
        or '채굴' in context:
        categories.append('05') # 석탄, 원유, 천연가스 광업
        
    if '금속' in context or '비철' in context or '철광석' in context \
        or '채굴' in context:
        categories.append('06') # 금속 광업
    
    if '석회석' in context or '점토' in context or '석재' in context \
        or '쇄석' in context or '모래' in context or '자갈' in context\
        or '천일염' in context or '염전' in context or '염업' in context\
        or '토사석' in context or '원료용' in context or '암염' in context\
        or '비금속광물' in context:
        categories.append('07') # 비금속 광업    
    
    

    if '사료' in context or '조제식품' in context or '식료품' in context \
        or '도시락' in context or '식사용' in context or '조미료' in context\
        or '식품' in context or '면류' in context or '설탕' in context\
        or '과자류' in context or '전분' in context or '당류' in context\
        or '낙농' in context or '식용' in context or '동물성' in context\
        or '식물성' in context or '곡물' in context or '과실' in context\
        or '채소' in context or '수산식물' in context or '육류' in context\
        or '도축' in context or '도축업' in context:
        
        if '중개' in context or '도매' in context:
            categories.append('463') # 상품 중개업
        else:
            categories.append('10') # 식료품 제조업

    if '음료' in context or '알코올' in context or '발효주' in context\
        or '탁주' in context or '약주' in context or '맥아' in context\
        or '맥주' in context or '증류주' in context or '합성주' in context\
        or '주정' in context or '소주' in context or '비알코올' in context\
        or '얼음' in context or '생수' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:        
            categories.append('11') # 음료 제조업

        
    if '담배' in context or '전자담배' in context or '담배제품' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('12') # 담배 제조업
        


    if '섬유제품' in context or '방적' in context or '방적업' in context \
        or '연사' in context or '가공사' in context or '직물' in context\
        or '직조' in context or '직물제품' in context or '면직물' in context\
        or '모직물' in context or '화학섬유직물' in context or '특수직물' in context\
        or '침구' in context or '자수' in context or '커튼' in context\
        or '모직물' in context or '화학섬유직물' in context or '특수직물' in context\
        or '천막' in context or '텐트' in context or '직물포대' in context\
        or '편조' in context or '원단' in context or '날염' in context\
        or '카펫' in context or '마루덮개' in context or '어망' in context\
        or '로프' in context or '세폭직물' in context or '세폭' in context\
        or '부직포' in context or '코드직물' in context or '표면처리' in context\
        or '적층' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('13') # 섬유제품 (의복 제외)
        
        
    if '의복' in context or '액세서리' in context or '모피' in context \
        or '봉제' in context or '겉옷' in context or '속옷' in context\
        or '잠옷' in context or '한복' in context or '봉제의복' in context\
        or '셔츠' in context or '블라우스' in context or '근무복' in context\
        or '작업복' in context or '가죽의복' in context or '유아용' in context\
        or '편조의복' in context or '편조' in context\
        or '스타킹' in context or '양말' in context or '모자' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('14') # 의복, 액세서리
            

    if '가죽' in context or '가방' in context or '신발' in context \
        or '핸드백' in context or '지갑' in context or '구두' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('15') # 가죽, 가방, 신발 제조업

            
    if '목재' in context or '나무제품' in context or '제재목' in context \
        or '박판' in context or '합판' in context or '적재판' in context\
        or '목제품' in context or '코르크' in context or '조물' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('16') # 목재, 나무제품 제조업

            
    if '펄프' in context or '종이' in context or '종이제품' in context \
        or '판지' in context or '신문용지' in context or '인쇄용' in context\
        or '필기용' in context or '원지' in context or '크라프트지' in context\
        or '골판지' in context:
        categories.append('17') # 펄프, 종이, 종이제품 제조업

    if '인쇄' in context or '기록매체' in context or '인쇄업' in context \
        or '제판' in context or '조판업' in context or '제책업' in context\
        or '인쇄관련' in context or '기록매체' in context or '복제업' in context:
        categories.append('18') # 인쇄 및 기록매체 복제업

    if '코크스' in context or '연탄' in context or '석유정제품' in context \
        or '정제처리' in context or '윤활유' in context or '정제물' in context:
        if '중개' in context:
            
            categories.append('46') # 상품 중개업
        else:
            categories.append('19') # 코크스, 연탄 및 석유정제품 제조업

    if '화학물질' in context or '화학제품' in context or '천연수지' in context \
        or '석탄화학계' in context or '화합물' in context or '유기화학' in context\
        or '무기화학' in context or '착색제' in context or '안료' in context \
        or '염료' in context or '유연제' in context or '산화물' in context \
        or '합성고무' in context or '플라스틱' in context or '살충제' in context \
        or '화학비료' in context or '복합비료' in context or ('농약' in context and '적발' not in context) \
        or '살균' in context or '상토' in context or '식물보호제' in context \
        or '페인트' in context or '코팅제' in context or '도료' in context \
        or '도포제' in context or '물감' in context or '세제' in context \
        or '화장품' in context or '치약' in context or '비누' in context \
        or '광택제' in context or '가향제' in context or '감광' in context \
        or '접착제' in context or '젤라틴' in context or '불꽃제품' in context \
        or '화학섬유' in context or '플라스틱' in context or '살충제' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('20') # 화학 물질 및 화학제품 제조업
        
        
    if '의료용' in context or '의약품' in context or '의약' in context \
        or '항생물질' in context or '제제' in context or '완제' in context\
        or '한의학' in context or '의학' in context or '한의약품' in context:
        categories.append('21') # 의료용 물질 및 의약품 제조업 
        
    if '고무제품' in context or '타이어' in context or '튜브' in context \
        or '비경화' in context or '고무' in context or '플라스틱' in context\
        or '필름' in context or '합성피혁' in context or '플라스틱제품' in context \
        or '봉투' in context or '성형제품' in context or '산화물' in context \
        or '합성고무' in context or '플라스틱' in context or '살충제' in context \
        or '화학비료' in context or '복합비료' in context or '농약' in context \
        or '살균' in context or '상토' in context or '식물보호제' in context \
        or '페인트' in context or '코팅제' in context or '도료' in context \
        or '도포제' in context or '물감' in context or '세제' in context \
        or '화장품' in context or '치약' in context or '비누' in context \
        or '광택제' in context or '가향제' in context or '감광' in context \
        or '접착제' in context or '젤라틴' in context or '불꽃제품' in context \
        or '화학섬유' in context or '플라스틱' in context or '살충제' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:
            categories.append('22') # 고무 및 플라스틱 제품 제조업
        
        
    if '판유리' in context or '유리' in context or '안전유리' in context \
        or '유리제품' in context or '유리용기' in context or '내화' in context\
        or '비내화' in context or '요업제품' in context\
        or '도자기' in context or '벽돌' in context or '블록' in context \
        or '시멘트' in context or '석회' in context or '플라스터' in context \
        or '모르타르' in context or '레미콘' in context or '콘크리트' in context \
        or '석제품' in context or '아스팔트' in context or '연마재' in context \
        or '암면' in context or '탄소섬유' in context or '분쇄물' in context:
        categories.append('23') # 비금속 광물제품 제조업
    
    if '제철업' in context or '제철' in context or '제강' in context \
        or '합금철' in context or '제강업' in context or '철강' in context\
        or '압연' in context or '압출' in context\
        or '압출제품' in context or '철강선' in context or '철강관' in context \
        or '주철관' in context or '강관' in context or '도금' in context \
        or '착색' in context or '강재' in context or '합금' in context \
        or '비철금속' in context or '비철' in context or '제련' in context \
        or '정련' in context or '주물' in context or '선철' in context\
        or '알루미늄' in context:
        
        if '중개' in context:
            categories.append('46') # 상품 중개업
        else:        
            categories.append('24') # 1차 금속 제조업
    
    
    if '금속제품' in context or '탱크' in context or '증기발생기' in context \
        or '판제품' in context or '공작물' in context or '구조재' in context\
        or '보일러' in context or '방열기' in context\
        or '핵반응기' in context or '증기보일러' in context or '무기' in context \
        or '총포탄' in context or '야금제품' in context or '단조제품' in context \
        or '압형제품' in context or '열처리업' in context or '도금업' in context \
        or '금속가공업' in context or '날붙이' in context or '파스너' in context \
        or '금속선' in context or '주방용기' in context or '용접봉' in context\
        or '피복' in context:
        categories.append('25') # 금속 가공제품 제조업
    
    if '반도체' in context or '전자직접회로' in context or '메모리용' in context \
        or '비메모리용' in context or '다이오드' in context or '트랜지스터' in context\
        or '발광' in context or '표시장치' in context or '인쇄회로기판' in context\
        or '실장기판' in context or '축전기' in context or '저항기' in context \
        or '전자카드' in context or '전자코일' in context or '컴퓨터' in context \
        or '모니터' in context or '프린터' in context or '방송장비' in context \
        or '통신장비' in context or '텔레비전' in context or '광학매체' in context \
        or '마그네틱' in context or '라디오' in context or '음향기기' in context\
        or '녹음' in context:
        categories.append('26') # 전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업
    
    if '방사선' in context or '의료용기기' in context or '정형외과' in context \
        or '보정' in context or '치과용' in context or '안경' in context\
        or '렌즈' in context or '측정' in context or '시험' in context\
        or '항해' in context or '제어' in context or '레이더' in context \
        or '전자기' in context or '제어장비' in context or '사진장비' in context \
        or '광학' in context or '사진기' in context or '영사기' in context \
        or '시계' in context:
        categories.append('27') # 의료, 정밀, 과학 기기 및 시계 제조업
    
    if '전동기' in context or '발전기' in context or '변압기' in context \
        or '안정기' in context or '저장장치' in context or '변환장치' in context\
        or '전기회로' in context or '접속장치' in context or '자동제어' in context\
        or '일차전지' in context or '축전지' in context or '절연선' in context \
        or '케이블' in context or '광섬유' in context or '도체' in context \
        or '전구' in context or '램프' in context or '조명' in context \
        or '조명장치' in context or '난방' in context or '경보'  in context\
        or '전기장비' in context:
        categories.append('28') # 전기 장비 제조업
    
    if '기계' in context or '내연기관' in context or '터빈' in context \
        or '유압' in context or '펌프' in context or '밸브' in context\
        or '압축기' in context or '구름베어링' in context or '동력' in context\
        or '트럭' in context or '승강기' in context or '컨베이어' in context \
        or '냉각' in context or '여과기' in context or '증류기' in context \
        or '사무용' in context or '동력식' in context:
        
        if '중개' in context or '도매' in context:
            categories.append('4653') # 산업용 기계 및 장비 도매업
        else:
            categories.append('29') # 기타 기계 및 장비 제조업
            
    
    if '자동차' in context or '트레일러' in context or '엔진' in context \
        or '승용차' in context or '엔진용' in context or '차체' in context\
        or '차체용' in context or '조향장치' in context or '현가장치' in context\
        or '제동장치' in context or '제동' in context or '오락용품' in context \
        or '장신용품' in context or '전달장치' in context or '전기장치'  in context\
        or '전기장비' in context or '모터사이클' in context or '타이어' in context\
        or '오토바이' in context or '스쿠터' in context:
        
        if '판매' in context or '소매' in context or '도매' in context \
            or '부품' in context or '내장품' in context or '중고' in context:
            if '중고' in context:
                categories.append('4522') # 자동차 중고 부품 및 내장품 판매업   
            elif '임대' in context:
                categories.append('761') # 운송장비 임대업 
            elif '모터사이클' in context:
                categories.append('4530') # 모터사이클 및 부품 판매업
            elif '부품' or '내장품' in context :
                categories.append('4521') # 자동차 신품부품 및 내장품 판매업 

                
        else:
            if '엔진' in context:
                if '신품' in context or '부품' in context:
                    categories.append('30310') # 자동차 엔진용 신품 부품 제조업
                else:
                    categories.append('30110') # 자동차용 엔진 제조업
                    
            elif '화물' in context or '특수' in context:
                categories.append('30122') # 화물 및 특수 목적 자동차 제조업
            elif '승용차' in context or '여객용' in context:
                categories.append('30121')  # 승용차 및 기타 여객용 자동차 제조업
            elif '차체' in context or '특장차' in context:
                if '신품' in context or '부품' in context:
                    categories.append('30310') # 자동차 차체용 신품 부품 제조업
                else:
                    categories.append('30201') # 차체 및 특장차 제조업 
            elif '구조' in context or '장치' in context or '변경' in context:
                categories.append('30202')  # 자동차 구조 및 장치 변경업
            elif '트레일러' in context or '세미' in context:
                categories.append('30203')  # 트레일러 및 세미 트레일러 제조업
            elif '동력' in context or '전달장치' in context:
                categories.append('30331') # 자동차용 신품 동력 전달장치 제조업
            elif '전기장치' in context:
                categories.append('30332') # 자동차용 신품 정기장치 제조업
            elif '조향장치' in context or '현가장치' in context\
                or '조향' in context or '현가' in context:
                categories.append('30391') # 자동차용 신품 조향장치 및 현가장치 제조업
            elif '제동' in context or '제동장치' in context:
                categories.append('30392') # 자동차용 신품 제동장치 제조업
            elif '의자' in context:
                categories.append('30393') # 자동차용 신품 의자 제조업
            elif '재제조' in context:
                categories.append('30400') # 자동차 재제조 부품 제조업   
            elif '모터사이클' in context or '오토바이' in context or '스쿠터' in context:
                categories.append('31920') # 모터사이클 제조업
            elif '환자' in context or '자전거' in context or '구급차' in context:
                categories.append('31991') # 자전거 및 환자용 차량 제조업
            elif '모터사이클' in context or '오토바이' in context or '스쿠터' in context:
                categories.append('31920') # 모터사이클 제조업    
            elif '자동차' in context:
                categories.append('30399') # 그 외 자동차용 신품 부품 제조업
            else:
                categories.append('31999') # 그 외 달리 분류되지 않은 운송장비 제조업

        
        
        
    
    if '운송장비' in context or '선박' in context or '보트' in context or '건조' in context \
        or '건조업' in context or '철도' in context or '기관차' in context or '차량' in context \
        or '기차' in context or '항공기' in context or '우주선' in context:
        
        if '선박' in context or '보트' in context or '건조' in context or '건조업' in context:
            if '오락' in context or '스포츠' in context or '스포츠용' in context:
                categories.append('31120') # 오락 및 스포츠용 보트 건조업
            elif '강선' in context:
                categories.append('31111') # 강선 건조업
            elif '합성수지선' in context or '합성수지' in context or '수지' in context or '합성' in context\
                or '수지선' in context:
                categories.append('31112') # 합성수지선 건조업
            elif '부분품' in context or '구성' in context:
                categories.append('31114') # 선박 구성 부분품 제조업
            else:
                categories.append('31113') # 기타 선박 건조업

        elif '철도' in context or '기관차' in context or '차량' in context or '기차' in context\
            or '철도장비' in context:
            if '기관차' in context or '차량' in context:
                categories.append('31201') # 기관차 및 기타 철도 차량 제조업
            else: 
                categories.append('30202') # 철도 차량 부품 및 관련 장치물 제조업

        elif '항공기' in context or '우주선' in context:
            if '유인' in context:
                categories.append('31311') # 유인 항공기, 항공 우주선 및 보조장치 제조업
            elif '무인' in context:
                categories.append('31312') # 무인 항공기 및 무인 비행장치 제조업
            elif '엔진' in context:
                categories.append('31321') # 항공기용 엔진 제조업
            elif '부품' in context:
                categories.append('31322') # 항공기용 부품 제조업
                
        elif '전투용' in context :
            categories.append('31910') # 전투용 차량 제조업

    
    if '가구' in context or '침대' in context or '매트리스' in context \
        or '소파' in context:
        categories.append('32') # 가구 제조업
    
    if '귀금속' in context or '장신용품' in context or '악기' in context \
        or '인형' in context or '장난감' in context or '오락용품' in context\
        or '회화용품' in context or '가발' in context or '모형' in context:
        categories.append('33') # 기타 제품 제조업
        
    if '수리업' in context or '수리' in context:
        categories.append('34') # 산업용 기계 및 장비 수리업
                
    if '전기' in context or '전기업' in context or '발전' in context \
        or '발전업' in context or '송전' in context or '배전' in context\
        or '배전업' in context or '연료용' in context or '배관공급업' in context\
        or '공기' in context or '배관' in context:
        categories.append('35') # 전기, 가스, 증기 및 공기 조절 공급업
        
    if '용수' in context or '생활용수' in context or '산업용수' in context \
        or '수도' in context or '수도업' in context:
        categories.append('36') # 수도업     
        
    if '하수' in context or '폐수' in context or '분뇨' in context:
        categories.append('37') # 하수, 폐수 및 분뇨 처리업      
        
    if '폐기물' in context or '해체' in context or '금속류' in context \
        or '비금속류' in context or '재생' in context or '재생업' in context:
        categories.append('38') # 폐기물 수집, 운반, 처리 및 원료 재생업
        
    if '정화' in context or '정화업' in context or '복원' in context \
        or '복원업' in context or '환경' in context:
        categories.append('39') # 환경 정화 및 복원업
        
    if '주택' in context or '아파트' in context or '비주거용' in context \
        or '지반조성' in context or '시설물' in context or '교량' in context\
        or '터널' in context or '철도' in context or '항만' in context\
        or '수로' in context or '환경설비' in context or '건설업' in context\
        or '조경' in context or '건설' in context:
        categories.append('41') # 종합 건설업
        
    if '기반조성' in context or '축조' in context or '구조물' in context \
        or '지반조성' in context or '시설물' in context or '교량' in context\
        or '보링' in context or '토공사업' in context or '그라우팅' in context\
        or '파일공사' in context or '전문공사' in context or '조적' in context\
        or '궤도' in context or '형틀' in context or '비계' in context\
        or '지붕' in context or '외벽' in context or '건물설비' in context\
        or '구축물' in context or '난방' in context or '소방시설' in context\
        or '전기배선' in context or '통신배선' in context or '도장' in context\
        or '도배' in context or '창호' in context or '미장' in context\
        or '타일' in context or '방수' in context or '공작물' in context\
        or '시설물' in context or '건설장비' in context:
        categories.append('42') # 전문직별 건설업
        
    
    if '운수' in context or '창고업' in context or '운송' in context \
        or '운송업' in context or '창고' in context or '화물' in context\
        or '여객' in context or '소화물' in context or '항공' in context\
        or '취급업' in context or '운송관련' in context or '운영업' in context\
        or '실내건축' in context or '건축마무리' in context:
        
        if '보관' in context:
            categories.append('5210') # 보관 및 창고업
        elif '서비스' in context:
            categories.append('529') # 그 외 기타 운송 관련 서비스업
        if '철도' in context or '소화물' in context \
            or '파이프라인' in context or '도로' in context: 
            categories.append('49') # 육상 운송 및 파이프라인 운송업
        elif '해상' in context or '수상' in context or '항만' in context \
            or '외항' in context or '내항' in context:
            categories.append('50') # 수상 운송업
        elif '항공' in context or '비행기' in context:
            categories.append('51') # 항공 운송업
        else:
            categories.append('52') # 창고 및 운송 관련 서비스업

    
    if '숙박' in context or '숙박업' in context\
        or '숙박시설' in context or '호텔' in context\
        or '여관' in context or '휴양' in context or '콘도' in context\
        or '민박' in context or '기숙사' in context or '고시원' in context:
        
            if '호텔' in context:
                categories.append('55101') # 호텔업
            elif '민박' in context:
                categories.append('55104') # 민박업
            elif '여관' in context:
                categories.append('55102') # 여관업
            elif '콘도' in context or '휴양' in context:
                categories.append('55103') # 휴양 콘도 운영업
            elif '기숙사' in context or '고시원' in context:
                categories.append('55901') # 기숙사 및 고시원 운영업
            else:
                categories.append('55909') # 그 외 기타 숙박업
    
    if '음식점' in context or '음식점업' in context or '음식' in context\
        or '식당' in context or '한식' in context or '요식' in context:
        
            if '한식' in context:
                categories.append('5611') # 한식 일반 음식점업
            elif '중식' in context or '일식' in context or '서양식' in context\
                or '외국' in context:
                categories.append('5612') # 외국식 음식점업
            elif '구내' in context or '구내식당' in context:
                categories.append('56130') # 기관 구내식당업
            elif '출장' in context or '이동' in context:
                categories.append('5614') # 출장 및 이동 음식점업
            else:
                categories.append('5619') # 기타 간이 음식점업
        
    if '주점' in context or '비알코올' in context or '음료' in context \
        or '유흥' in context:
            
            if '맥주' in context or '주점' in context or '유흥' in context:
                categories.append('5621') # 주점업
            elif '비알코올' in context or '커피' in context:
                categories.append('5622') # 비알코올 음료점업
            else:
                categories.append('56229') # 기타 비알코올 음료점업
        
        
    if '출판' in context or '서적' in context or '인쇄물' in context or '교과서' in context\
        or '간행물' in context or '신문' in context or '잡지' in context\
        or '만화'in context:
            categories.append('581') # 서적, 잡지 및 기타 인쇄물 출판업

    if '소프트웨어' in context or '게임' in context\
        or '블록체인' in context :
            categories.append('582') # 소프트웨어 개발 및 공급업
        
    if '영상' in context or '비디오물' in context or '영화관' in context\
        or '배급업' in context or '제작업' in context or '방송' in context\
        or '영화' in context:
            categories.append('591') # 영화, 비디오물, 방송 프로그램 제작 및 배급
        
    if '오디오물' in context or '녹음' in context or '오디오' in context\
        or '녹음시설' in context:
            categories.append('5920') # 오디오물 출판 및 원판 녹음업
            
    if '방송' in context or '방송업' in context:
        if '라디오' in context:
            categories.append('60100') # 라디오 방송업
        if '텔레비전' in context or '지상파' in context:
            categories.append('602') # 텔레비전 방송업
        

    if '우편' in context or '통신' in context:
        categories.append('61') # 우편 및 통신업
    
    if '컴퓨터' in context or '컴퓨터시설' in context or '프로그래밍' in context\
        or '시스템' in context:
        categories.append('62') # 컴퓨터 프로그래밍, 시스템 통합 및 관리업
    
    if '인터넷' in context or '호스팅' in context or '포털' in context\
        or '자료처리' in context:
            categories.append('631') # 자료 처리, 호스팅, 포털 인터냇 매개 서비스업
    
    if '뉴스' in context:
        categories.append('63910') # 뉴스 제공업
    if '데이터베이스' in context:
        categories.append('63991') # 데이터베이스 및 온라인 정보 제공업
    if '정보서비스' in context:
        categories.append('63999') # 그외 기타 정보 서비스업
        
        
    if '금융' in context or '금융업' in context or '은행' in context \
        or '중앙은행' in context or '일반은행' in context or '신용조합' in context\
        or '저축' in context or '신용' in context:
        categories.append('641') # 은행 및 저축기관
        
    if '신탁' in context\
        or '신탁업' in context or '투자' in context or '투자업' in context:
        categories.append('642') # 신탁업 및 집합 투자업
        
    if '여신' in context or '리스업' in context or '신용카드' in context\
        or '기금' in context or '지주회사' in context or '할부' in context:
        categories.append('649') # 기타 금융업
        
    if '보험' in context or '보험업' in context or '연금' in context\
        or '연금업' in context or '생명보험' in context or '보증보험' in context\
        or '건강보험' in context or '산업재해' in context\
        or '재보험' in context or '공제업' in context:
        categories.append('65') # 보험 및 연금업
        
    if '금융' in context or '증권' in context or '선물' in context \
        or '거래' in context or '투자' in context or '자문' in context\
        or '일임업' in context or '손해' in context or '중개업' in context\
        or '사정업' in context:
        categories.append('66') # 금융 및 연금관련 서비스업
        
    if '부동산' in context or '부동산업' in context or '임대' in context \
        or '주거용' in context or '비주거용' in context\
        or '부동산중개' in context or '주택' in context:
        if '임대' in context:
            categories.append('6811') # 부동산 임대업
        elif '개발' in context or '공급' in context:
            categories.append('6812') # 부동산 개발 및 공금업
        elif '관리' in context:
            categories.append('6821') # 부동산 관리업
        elif '중개' in context or '자문' in context or '감정' in context:
            categories.append('6822') # 부동산 중개, 자문 및 감정 평가업
        
    if '개발' in context or '기술' in context or '연구' in context \
        or '산업' in context or '에너지사용량' in context or '엔지니어링' in context\
        or '대학' in context or '자연과학' in context or '공학' in context\
        or '사회과학' in context or '법무' in context or '회계' in context\
        or '세무' in context or '광고' in context or '광고업' in context\
        or '시장조사' in context or '여론조사' in context or '여론' in context\
        or '컨설팅' in context or '경영' in context or '설계' in context\
        or '측량' in context or '지질' in context or '수의업' in context\
        or '디자인업' in context or '사진' in context or '촬영' in context:
        
        if '물리' in context or '화학' in context or '생물' in context:
            categories.append('70111') # 물리, 화학 및 생물학 연구개발업
        elif '농림' in context or '수의학' in context or '동물' in context\
            or '수의업' in context:
            categories.append('70112') # 농림수산학 및 수의학 연구개발업
        elif '의학' in context or '약학' in context:
            categories.append('70113') # 의학 및 약학 연구개발업
        elif '전기' in context or '전자' in context:
            categories.append('70121') # 전기 전자공학 연구개발업
        elif '자연' in context:
            categories.append('70119') # 기타 자연과학 연구개발업
        elif '공학' in context and '연구' in context:
            categories.append('70129') # 기타 공학 연구개발업
        elif '경제' in context or '경영' in context:
            categories.append('70201') # 경제 및 경영학 연구개발업
        elif '사회과학' in context:
            categories.append('70209') # 기타 인문 및 사회과학 연구개발업
            
    if '변호' in context or '변리' in context or '법무' in context:
        categories.append('7110') # 법무관련 서비스업
    if '회계' in context or '세무' in context:
        categories.append('7120') # 회계 및 세무관련 서비스업
    if '광고업' in context or '광고물' in context:
        categories.append('713') # 광고업
    if '시장조사' in context or '여론' in context or '조사업' in context:
        categories.append('71400') # 시장조사 및 여론 조사업
    if '본부' in context or '회사본부' in context:
        categories.append('7151') # 회사 본부
    if '컨설팅' in context or '공공관계' in context:
        categories.append('7153') # 경영 컨설팅 및 공공 관계 서비스업
    
    if '전문' in context and '서비스' in context:
        categories.append('71600') # 기타 전문 서비스업
        
    if ('서비스' in context and ('건축' in context or '기술' in context or '엔지니어링' in context))\
        or '조경' in context or '측량' in context\
        or '탐사' in context or '지질' in context:
        categories.append('729') # 건축 기술, 엔지니어링 및 기타 과학기술 서비스업
        
    if '수의업' in context:
        categories.append('73100') # 수의업
    if '디자인' in context:
        if '인테리어' in context:
            categories.append('73201') # 인테리어 디자인
        elif '제품' in context:
            categories.append('73202') # 제품 디자인업
        elif '시각' in context:
            categories.append('73203') # 시각 디자인업
        else:
            categories.append('73209') # 패션, 섬유류 및 기타 전문 디자인업
    
    if '사진' in context and '촬영' in context:
        categories.append('7330') # 사진 촬영 및 처리업
    if '매니저' in context:
        categories.append('73901') # 매니저업
    if '번역' in context or '통역' in context:
        categories.append('73902') # 번역 및 통역 서비스업
    if '재산권' in context and '중개' in context:
        categories.append('73903') # 사업 및 무형 재산권 중개업
    if '감정' in context and '물품' in context or '계량' in context or '견본' in context:
        categories.append('73904') # 물품감정, 계량 및 견본 추출업
    
        categories.append('7320') # 전문 디자인업
        
        

        
        
        
    N1 = ['사업시설', '조경', '방제', '유지관리', '청소업', '소독', '조경관리']
    
    for i in N1:
        if i in context and '74' not in categories:
            categories.append('74') # 사업시설 관리 및 조경 서비스업
            
            
    N2 = ['고용', '알선', '일용', '인력', '인사관리', '여행사', '여행사업', '경비', '예약',\
         '경호', '탐정', '보안', '복사', '사무지원', '콜센터', '텔레마케팅', '전시', '컨벤션',\
         '행사', '추심', '포장', '충전']
    
    for i in N2:
        if i in context and '75' not in categories:
            categories.append('75') # 사업 지원 서비스업
            
            
    if '임대' in context:
        if '스포츠' in context or '레크리에이션' in context:
            categories.append('76210') # 스포츠 및 레크리에이션 용품임대업
        elif '음반' in context or '비디오' in context:
            categories.append('76220') # 음반 및 비디오물 임대업
        elif '서적' in context or '책' in context:
            categories.append('76291') # 서적 임대업
        elif '의류' in context:
            categories.append('76292') # 의류 임대업
        elif '토목' in context or '건설' in context:
            categories.append('7631')
        elif '컴퓨터' in context or '기계' in context or '사무용' in context:
            categories.append('7632')
        elif '산업' in context:
            categories.append('7638')
        elif '재산권' in context and '무형' in context:
            categories.append('76400')
            
            
            
            
            
    administration = ['행정', '국방', '사회보장', '입법', '정부기관', '사회서비스',\
                      '노동', '외무', '공공질서', '사법']
        
    for i in administration:
        if i in context and '84' not in categories:
            categories.append('84')
            
            
    education = ['교육기관', '초등', '중등', '고등', '특수학교', '외국인학교',\
                '대안학교', '초등학교', '중학교', '고등학교', '교습학원'\
                '예술학원', '외국어학원', '학원', '훈련기관', '훈련학원']
        
    for i in education:
        if i in context and '교육업' not in categories:
            categories.append('85') 
    
    
            
    Q1 = ['보건', '보건업', '사회복지', '병원', '의원', '의료업',\
                '공중보건', '의료']
    Q2 = [ '복지시설', '운영업', '보육시설', '사회복지']
        
    for i in Q1:
        if i in context and '86' not in categories:
            categories.append('86') 
    for i in Q2:
        if i in context and '87' not in categories:
            categories.append('87') 
    
    leisure = ['예술', '스포츠', '여가', '창작', '공연시설', '공연',\
                '도서관', '사적지', '박물관', '식물원', '동물원']
              
    leisure2 = ['오락', '경기장', '골프장', '스포츠시설', '유원지'\
              '낚시', '오락장', '수상오락', '사행시설', '무도장'\
              '체육공원', '복권', '낚시장', '노래방', '게임방'\
              'PC방', '경주장', '볼링장', '당구장', '연습장', '스키장']
        
    for i in leisure:
        if i in context and '90' not in categories:
            categories.append('90') 
    
    for i in leisure2:
        if i in context and '91' not in categories:
            categories.append('91') 
            
    org1 = ['협회', '단체', '노동조합', '종교', '시민운동']
    org2 = ['수리', '수리업']
    org3 = ['미용', '미용업', '마사지', '욕탕'\
              '세탁', '세탁업', '장례식장', '화장터', '간병'\
              '장묘', '점술', '예식장', '사행시설', '무도장'\
              '체육공원', '복권', '낚시장', '노래방', '게임방'\
              'PC방', '경주장', '볼링장', '당구장', '연습장', '스키장']
        
    for i in org1:
        if i in context and '94' not in categories:
            categories.append('94') 
    
    for i in org2:
        if i in context and '95' not in categories:
            categories.append('95') 
            
    for i in org3:
        if i in context and '96' not in categories:
            categories.append('96') 
            
    international = ['주한', '외국기관', '국제', '외국']
        
    for i in international:
        if i in context and '99' not in categories:
            categories.append('99') 
            
    inhouse = ['요리사', '가정부', '세탁부', '보모', '유모', '비서', '집사', '운전사', '관리원', '가정교사']
    for i in inhouse:
        if i in context and '97000' not in categories:
            categories.append('97000') 
            
    if len(categories) == 0:
        return '98'
    else:
        return ', '.join(categories)
    

df['추출_주요업종'] = df['context_tokens'].apply(small_category)
    


In [12]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', '정책ID', '정책서비스ID', '서비스명', '소관기관',
       '소관기관유형', '서비스목적', '지원대상', '지원내용', '신청절차', '선정기준', '신청기한_시작', '신청기한_끝',
       '상세조회URL', '문의처전화번호', '온라인신청사이트URL', '지원유형', '추출_시도', '추출_시군구',
       '추출_종업원수_최소', '추출_종업원수_최대', '추출_기업규모', '추출_주요업종', '추출_선호특성',
       '추출_설립기간_최소', '추출_설립기간_최대', '추출_시도1', '추출_시군구1', 'context',
       'context_tokens'],
      dtype='object')

In [13]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1','context','context_tokens'], axis=1, inplace=True)

In [15]:
df.to_csv('policy_final_01192022.csv')

## 관심특성

In [ ]:
# 관심 특성 분류

# 위에 사용한 context_tokens 콜럼 사용 - 추후 수정 예정

def characteristic(context):
    if '장애인' in context or '장애' in context:
        return '장애인'
    elif '창업' in context:
        return '창업'
    elif '친환경' in context or '재활용' in context or '환경보호' in context\
        or '신재생' in context:
        return '친환경'
    elif '영화' in context or '음악' in context or '예술' in context:
        return '문화'
    elif '코로나' in context:
        return '코로나'
    else:
        return '기타'



## 설립기간

In [ ]:
# 설립 기간 분류 함수

def founded_date(context):
    if '석사' in context or '박사' in context or '근로자' in context:
        return []
    elif '지원할 수 없음' in context or '제외 대상' in context \
        or '직무발명' in context:
        return []
    else:

        if '년' in context:
            if context[context.find('년')+2] == '~':
                remains = ''
            
                start = int(context[context.find('년')-2: context.find('년')])
                remains = context[context.find('년')+1:]
                
                end = int(remains[remains.find('년')-2: remains.find('년')])
                return [start, end]
            
            elif context[context.find('년')-1] == '전':
                return []
            
            elif context[context.find('년')+2:context.find('년')+4] == '이상':
                if context[context.find('년')-2]:
                    try:
                        start = int(context[context.find('년')-2: context.find('년')])
                    except:
                        if context[context.find('년')-1]:
                            try:
                                start = int(context[context.find('년')-1: context.find('년')])
                            except:
                                return []
                                
                end = '-'
                return [start, end]
            
            elif context[context.find('년')+2:context.find('년')+4] == '미만'\
                or context[context.find('년')+2:context.find('년')+4] == '이내'\
                or context[context.find('년')+2:context.find('년')+4] == '이하'\
                or context[context.find('년')+1] == '간'\
                or context[context.find('년')+1:context.find('년')+4] == '이 경과하지 않은':
                start = 0
                
                try:
                    end = int(context[context.find('년')-2: context.find('년')])
                except:
                    if context[context.find('년')-1]:
                        try:
                            end = int(context[context.find('년')-1: context.find('년')])
                        except:
                            return []
                return [start, end]
            
            try:
                num = int(context[context.find('년')-2: context.find('년')])
            except:
                return []
            return []
        else:
            return []


# 두가지 콜럼으로 분류

def foundate1(list):
    if len(list)<1:
        return 0
    else:
        return list[0]

def foundate2(list):
    if len(list)<1:
        return 9999999
    else:
        return list[1]


## 지역

### (((개체명 인식하기 - 실행할필요X)))
- (개체명인식 결과를 데이터프레임으로 빼내서 불러옴)
- 이 부분 실행안해도 됨
- 지역명 딕셔너리부터 실행
- 주어진 정책데이터에서 태그를 추출하기 위한 전단계 과정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## ★★ 주의!!! (시간 조금 걸림)  ★★ ## 1팀 폴더내에 KoBertTokenizer.py 와 KoBERTmodel_1.pt 필요
# Hugging Face의 transformers 모델을 설치
!pip install "transformers==3.4.0"

In [ ]:
# KoBertTokenizer import 위한 경로설정
import sys
sys.path.append('/content/drive/MyDrive/1팀')
print(sys.path)

In [ ]:
import tensorflow as tf
import torch
import re
from transformers import BertModel
from transformers import BertTokenizer
from tokenization_kobert import KoBertTokenizer
from transformers import BertForTokenClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import time
import datetime
import random

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# GPU 디바이스 설정
device_name = tf.test.gpu_device_name()
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
## ★★ 실행주의!!! (오래걸림 - 빠르면 1시간 느리면 3시간)  ★★ ## 

# 모델 불러오기
model = torch.load('/content/drive/MyDrive/1팀/KoBERTmodel_1.pt')


# 숫자값:태그
tag_dict = {'[PAD]' : 0, '[CLS]': 1, '[SEP]': 2, 'O' : 3,
            'B-PER': 4, 'I-PER': 5, 
            'B-ORG': 6, 'I-ORG': 7,
            'B-LOC': 8, 'I-LOC': 9, 
            'B-POH': 10, 'I-POH': 11,
            'B-DAT': 12, 'I-DAT': 13,
            'B-TIM': 14, 'I-TIM': 15,
            'B-DUR': 16, 'I-DUR': 17,
            'B-MNY': 18, 'I-MNY': 19,
            'B-PNT': 20, 'I-PNT': 21, 
            'B-NOH': 22, 'I-NOH': 23}
tag_dict_decode = inv_map = {v: k for k, v in tag_dict.items()}





# 입력 데이터 전처리
def convert_input_data(sentences):
    text_CLS = ["[CLS] " + str(txt) + " [SEP]" for txt in sentences]
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    # 토크나이징
    tokenized_texts = [tokenizer.tokenize(sent) for sent in text_CLS]
    MAX_LEN = 128 # MAX_LEN 설정
    # 정수 인코딩 및 패딩 진행
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # 어텐션 마스크 설정
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

# 문장 테스트
def test_sentences(sentences):
    stopFlag = False
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    # 평가모드로 변경
    model.eval()
    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)
    if torch.cuda.is_available():    
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask
                        )
    logits = np.argmax(outputs[0].to('cpu').numpy(), axis=2)
    
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(b_input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, logits[0]):
        if stopFlag == True:
          break
        if token == '[SEP]': # 패딩 전까지의 출력만을 보기 위해
          stopFlag = True
        new_labels.append(tag_dict_decode[label_idx])
        new_tokens.append(token)

    # return logits
    return new_labels, new_tokens


# 출력 결과 가공
def inferencing(input_text, new_label, new_token):
  all_inf_list = [] # inf_list들로 이루어진 리스트
  inf_list = [] # [태그, 태그에 대응하는 단어]로 이루어진 리스트
  isB = False # B- 태그가 나왔을 경우 true로 설정
  tagword = "" # 토큰을 하나씩 연결하여 태그에 대응하는 하나의 단어로 만들기 위한 문자열

  for label, token in zip(new_label, new_token):
    notBFlag = False # B- 없이 I-만 나온 애들 처리하기 위함
    # B- 태그가 나왔을 경우
    if "B-" in label:
      # 만일 B- 태그가 나온상태에서 또 B- 태그가 나왔을 경우 (개체명이 연속으로 등장했을 경우)
      if isB:
        inf_list.append(tagword) 
        all_inf_list.append(inf_list)
        inf_list=[]
        tagword = ""
      # 태그명을 inf_list에 추가 ex) ORG, PER
      inf_list.append(label[2:5])
      
      if "▁" in token: # 만일 B- 태그에 대응하는 토큰이 ▁으로 시작할 경우
        tagword += token.replace("▁","") # '▁' 지움
      else: # 아닐 경우
        tagword += token # 그냥 tagword에 추가

      isB = True # B- 태그가 나왔으니 True로 설정
    
    # B- 태그가 나온 상태고, I- 태그가 나왔을 때 (B- 태그와 연결되지 않은 I- 태그는 비정상으로 판단)
    elif isB and "I-" in label:
      
      if label[2:5] not in inf_list:
        notBFlag = True
      if notBFlag == False:
        if "▁" in token: # 만일 I- 태그에 대응하는 토큰이 ▁으로 시작할 경우
          tagword_before = ""
          tagword_before += token.replace("▁","") # '▁' 지우고 tagword에 추가
          tagword += tagword_before.replace(tagword_before, " "+tagword_before)
        else: # 아닐 경우
          tagword += token

    # B- 태그가 나온 상태고, O 태그, 혹은 끝났을 때 (개체명이 끝났을 때)
    elif isB and ("O" in label or "[SEP]" in label):
      inf_list.append(tagword) # 지금까지의 tagword를 inf_list에 추가
      all_inf_list.append(inf_list) # inf_list를 all_inf_list에 추가
      inf_list=[] # 싹 다 초기화
      tagword = ""
      isB = False
  
  # 예쁘게 출력하기
  for text_tag in all_inf_list:
    if text_tag[1] in input_text:
      input_text = input_text.replace(text_tag[1], "<"+text_tag[1]+":"+text_tag[0]+">")

  return input_text


# 지원대상 개체명인식
policy_df['지원대상_개체명'] = ""
for i in range(len(policy_df['지원대상'])):  # 서비스명, 지원대상에서 지역추출  # 소관기관명 : 피쳐엔지니어링으로 추출
    input_text = policy_df['지원대상'][i]
    new_label, new_token = test_sentences([input_text])
    infer = inferencing(input_text, new_label, new_token)
    policy_df['지원대상_개체명'][i] = infer

# 선정기준 개체명 인식
policy_df['선정기준_개체명'] = ""
for i in range(len(df['선정기준'])):  # 선정기준, 지원대상에서 지역추출  # 소관기관명 : 피쳐엔지니어링으로 추출
    input_text = df['선정기준'][i]
    new_label, new_token = test_sentences([input_text])
    infer = inferencing(input_text, new_label, new_token)
    policy_df['선정기준_개체명'][i] = infer


# CSV파일로 저장하기
policy_df.to_csv("/content/drive/MyDrive/1팀/wello_220113.csv", index = False)

### 지역명 딕셔너리

In [ ]:
# 지역명 태그 딕셔너리 만들기

# 1) 시도 태그딕셔너리
policy_tag = pd.read_excel("/content/drive/MyDrive/1팀/정책데이터 태그셋리스트(AIB전달용).xlsx",sheet_name='상위코드', header=1 )
policy_tag = policy_tag.iloc[15:, 4:7]
policy_tag.columns = ['지역(시도)', '번호', '상위코드']
policy_tag.reset_index(drop=True, inplace=True)
region_tag = policy_tag[0:18]
region_tag = region_tag[['지역(시도)','상위코드']]

# 단어와 개체명 정보끼리 딕셔너리 만들기
region_zip = {w: t for w, t in zip(region_tag["지역(시도)"].values.tolist(), region_tag["상위코드"].values.tolist())}
# region_zip_reverse = {t:w for w, t in region_zip.items()}

# 지역이 축약되는 경우도 많으므로 적절하게 처리해서 row 추가 
a = 18
for i in region_tag['지역(시도)']:
    if i == '충청북도':
        region_tag.loc[a] = ['충북', region_zip[i]]
        a+=1
    elif i == '충청남도':
        region_tag.loc[a] = ['충남', region_zip[i]]
        a+=1
    elif i == '전라북도':
        region_tag.loc[a] = ['전북', region_zip[i]]
        a+=1
    elif i == '전라남도':
        region_tag.loc[a] = ['전남', region_zip[i]]
        a+=1
    elif i == '경상북도':
        region_tag.loc[a] = ['경북', region_zip[i]]
        a+=1
    elif i == '경상남도':
        region_tag.loc[a] = ['경남', region_zip[i]]
        a+=1
    else:
        region_tag.loc[a] = [i[0:2], region_zip[i]]
        a+=1

# 단어와 개체명 정보끼리 딕셔너리 만들기
region_zip = {w: t for w, t in zip(region_tag["지역(시도)"].values.tolist(), region_tag["상위코드"].values.tolist())}
# region_zip_reverse = {t:w for w, t in region_zip.items()}

# 2) 시군구 딕셔너리 만들고 추가
policy_low_tag= pd.read_excel("/content/drive/MyDrive/1팀/정책데이터 태그셋리스트(AIB전달용).xlsx",sheet_name='하위코드', header=1 )
region_low_tag = policy_low_tag.iloc[1:247,[4,6]].dropna(how='all')
region_low_tag.columns = ['지역(시군구)', '하위코드']
region_low_tag=region_low_tag.reset_index(drop=True)

# 단어와 개체명 정보끼리 쌍으로 묶기. ('전체'는 여러가지라서 따로 처리가 필요)
region_low_tag = region_low_tag[region_low_tag['지역(시군구)'] !='전체']
region_low_zip = {w:t for w, t in zip(region_low_tag["지역(시군구)"].values.tolist(), region_low_tag["하위코드"].values.tolist())}
# region_low_zip_reverse = {t:w for w, t in region_low_zip.items()}

# 지역이 축약되는 경우도 많으므로 적절하게 처리해서 row 추가 
a = 244
for i in region_low_tag['지역(시군구)']:
    if len(i[:-1]) == 1:
        continue
    elif i == '세종시 전체':
        continue
    elif i == '제주시':
        continue
    else:
        region_low_tag.loc[a] = [i[:-1], region_low_zip[i]]
        a+=1

region_low_zip = {w:t for w, t in zip(region_low_tag["지역(시군구)"].values.tolist(), region_low_tag["하위코드"].values.tolist())}
# region_low_zip_reverse = {t:w for w, t in region_low_zip.items()}

# 두 파이썬 딕셔너리를 병합(시도+시군구)
region_zip.update(region_low_zip)

### 소관기관 추출(피쳐엔지니어링)

In [ ]:
# CSV파일 읽기 - 개체명 인식컬럼이 추가된 정책metetable
df_meta = pd.read_csv("/content/drive/MyDrive/1팀/wello_220113.csv")

In [ ]:
# 소관기관에서 지역명 추출
df_meta['소관기관 추출_시도'] = ""
df_meta['소관기관 추출_시군구'] = ""

# 한 셀 안에서 여러개의 정보 쉼표 기준으로 나누고 딕셔너리의 value를 찾는 함수 작성
def Extract (data_x):
    li_high = []  # 시도
    li_low = []   # 시군구

    for text in data_x.split(' '):
        if text in region_zip: 
            if len(region_zip[text]) == 6 :
                li_high.append(region_zip[text])
            else:
                li_low.append(region_zip[text])
        else:
            continue
    
    high = ", ".join(list(set(li_high)))  # 중복값 제거 후 문자열만 추출
    low = ", ".join(list(set(li_low)))

    return high, low

for i in range(len(df_meta['소관기관'])):
    high, low = Extract(df_meta['소관기관'][i])

    df_meta['소관기관 추출_시도'][i] = high
    df_meta['소관기관 추출_시군구'][i] = low

### 지원대상 추출(개체명인식)

In [ ]:
# 소관기관에서 추출 불가능한 지역 : 지원대상에서 추출

# 한 셀 안에서 여러개의 정보 쉼표 기준으로 나누고 딕셔너리의 value를 찾는 함수 작성
def for_moon(data_x):
    li_high = []  # 시도
    li_low = []   # 시군구
    no_category = []   # 시도, 시군구에 분류되지 않는 하위지역

    for text in data_x.split(', '):
        if text in region_zip: 
            if len(region_zip[text]) == 6 :
                li_high.append(region_zip[text])
            else:
                li_low.append(region_zip[text])
        else:
            if text =='수도권':
                li_high.extend([region_zip['서울'],region_zip['경기']])
            elif text =='한국':
                li_high.append(region_zip['전국'])
            elif text =='개성공업지구':
                continue
            else:
                if len(text) == 1:   # 한 글자는 제거
                    continue
                else:
                    no_category.append(text)
    
    high = ", ".join(list(set(li_high)))  # 중복값 제거 후 문자열만 추출
    low = ", ".join(list(set(li_low)))
    no = ", ".join(list(set(no_category)))
        
    return high, low, no

import re

df_meta['지원대상 추출_시도'] = ""
df_meta['지원대상 추출_시군구'] = ""
df_meta['지원대상 추출_말단행정구역']= ""

finding = re.compile(r"<[^>]+:LOC>")  
for i in range(len(df_meta['지원대상_개체명'])):
  if df_meta['지원대상_개체명'][i].find('LOC') != -1:
    match_object = finding.findall(df_meta['지원대상_개체명'][i])  #리스트로 반환
    match_object = ', '.join(match_object)
    text = re.sub('[<>:LOC]', '',match_object)  # 한글 이외에 제외.
    high, low, no = for_moon(text)

    df_meta['지원대상 추출_시도'][i] = high
    df_meta['지원대상 추출_시군구'][i] = low
    df_meta['지원대상 추출_말단행정구역'][i] = no

### 소관기관+지원대상
- 다르게 추출된 부분에 대한 비교 필요

In [ ]:
# 소관기관, 지원대상 데이터 합치기
df_meta['추출_시도'] = ''
df_meta['추출_시군구'] =''
for i in range(len(df_meta)):
    if df_meta['소관기관 추출_시도'][i] == '':  # 먄약 소관기관에서 추출한 데이터가 없다면,
        df_meta['추출_시도'][i] = df_meta['지원대상 추출_시도'][i]  # 지원대상에서 추출한 데이터로 대체
    else:
        df_meta['추출_시도'][i] = df_meta['소관기관 추출_시도'][i]

for i in range(len(df_meta)):
    if df_meta['소관기관 추출_시군구'][i] == '': 
        df_meta['추출_시군구'][i] = df_meta['지원대상 추출_시군구'][i]
    else:
        df_meta['추출_시군구'][i] = df_meta['소관기관 추출_시군구'][i]

### 선정기준 추출(개체명 인식)

In [ ]:
# 선정기준에서 지역명 추출

df_meta['선정기준_개체명'] = df_meta['선정기준_개체명'].fillna("")
import re

df_meta['선정기준 추출_시도'] = ""
df_meta['선정기준 추출_시군구'] = ""
df_meta['선정기준 추출_말단행정구역']= ""

finding = re.compile(r"<[^>]+:LOC>") 
for i in range(len(df_meta['선정기준_개체명'])):
  if df_meta['선정기준_개체명'][i].find('LOC') != -1:
    match_object = finding.findall(df_meta['선정기준_개체명'][i])  #리스트로 반환
    match_object = ', '.join(match_object)
    text = re.sub('[<>:LOC]', '',match_object)  # 한글 이외에 제외.
    high, low, no = for_moon(text)

    df_meta['선정기준 추출_시도'][i] = high
    df_meta['선정기준 추출_시군구'][i] = low
    df_meta['선정기준 추출_말단행정구역'][i] = no

### 소관기관+지원대상+선정기준
- 다르게 추출된 부분에 대한 비교분석 필요

In [ ]:
for i in range(len(df_meta)):
    if df_meta['추출_시도'][i] == '':  # 만약 기존에 추출한 데이터가 없다면,
        df_meta['추출_시도'][i] = df_meta['선정기준 추출_시도'][i]  # 선정기준에서 추출한 데이터로 대체

for i in range(len(df_meta)):
    if df_meta['추출_시군구'][i] == '': 
        df_meta['추출_시군구'][i] = df_meta['선정기준 추출_시군구'][i]

### policy_df 에 적용

In [ ]:
# policy_df['시도'] = df_meta['추출_시도']
# policy_df['시군구'] = df_meta['추출_시군구']

## 종업원수 추출

In [ ]:
# # CSV파일 읽기 - 개체명 인식컬럼이 추가된 정책metetable
# df_meta = pd.read_csv("/content/drive/MyDrive/1팀/wello_220113.csv")

### 컬럼별 추출

In [ ]:


# 지원대상에서 추출
def support_target(sentence) : 
    finding = re.compile(r"(<[0-9]+[인명]:NOH>\s이상|<[0-9]+[인명]:NOH>\s이하|<[0-9]+[인명]:NOH>\s미만|<[0-9]+[인명]:NOH>\s사업주|<[0-9]+[인명]:NOH>\s창조기업)") 
    if sentence.find('NOH') != -1:
        match_object = finding.findall(sentence)  # 리스트로 반환
        match_object = ', '.join(list(set(match_object)))
        text = re.sub('[<>:NOH]', '',match_object) 
        return text

# 선정기준에서  추출
df_meta['선정기준_개체명']=df_meta['선정기준_개체명'].fillna("")
def selection_standard(sentence) : 
    finding = re.compile(r"(<[0-9]+[인명]:NOH>\s이상|<[0-9]+[인명]:NOH>\s이하|<[0-9]+[인명]:NOH>\s미만|<[0-9]+[인명]:NOH>\s사업주|<[0-9]+[인명]:NOH>\s창조기업)")  
    if df_meta['선정기준_개체명'][i].find('NOH') != -1:
        match_object = finding.findall(df_meta['선정기준_개체명'][i])  # 리스트로 반환
        match_object = ', '.join(list(set(match_object)))
        text = re.sub('[<>:NOH]', '',match_object)  # 한글 이외에 제외.
        return text

df_meta['추출_종업원수']=df_meta['지원대상_개체명'].apply(support_target)
df_meta['추출_종업원수1']=df_meta['선정기준_개체명'].apply(support_target)

# 적절한 값으로 대체
df_meta['추출_종업원수'][4454] = '5인 이상'
df_meta['추출_종업원수1'][5715] = '5인 이상'
df_meta['추출_종업원수1'][6256] = '5인 이상'
df_meta['추출_종업원수1'][7963]  = '10인 미만'
df_meta['추출_종업원수1'][8453]  = '5인 이상'
# 같은 값일 경우 작은 명수 기준. 

# merge
for i in range(len(df_meta['추출_종업원수'])):
    if df_meta['추출_종업원수'][i] =="":
        df_meta['추출_종업원수'][i] = df_meta['추출_종업원수1'][i]

### 추출된 종업원에서 최소, 최대값 추출

In [ ]:
df_meta['추출_종업원수_최소']= ""
df_meta['추출_종업원수_최대']= ""

# 최소 종업원수, 최대 종업원수 숫자로 추출('이상', '이하'를 기준으로 함)
for i in range(len(df_meta)):
    if df_meta['추출_종업원수'][i]:
        for text in df_meta['추출_종업원수'][i].split(', '):
            if '이상' in text :
                df_meta['추출_종업원수_최소'][i] = int(re.sub(r'[^0-9]', '', text))  # 숫자만 추출
            elif '이하' in text:
                df_meta['추출_종업원수_최대'][i] = int(re.sub(r'[^0-9]', '', text))
            elif '미만' in text:
                df_meta['추출_종업원수_최대'][i] = int(re.sub(r'[^0-9]', '', text))-1
            elif ('사업주' or '창조기업') in text:
                df_meta['추출_종업원수_최대'][i] = int(re.sub(r'[^0-9]', '', text))
                df_meta['추출_종업원수_최소'][i] = int(re.sub(r'[^0-9]', '', text))
            else:  # 이미 추출되어 숫자만 있을 경우
                continue
    else:  # 추출된 종업원수가 없을 경우 0으로 채움
        df_meta['추출_종업원수_최소'][i] = 0
        df_meta['추출_종업원수_최대'][i] = 0


# 결측치 or 빈 값 처리
df_meta['추출_종업원수_최소'] = df_meta['추출_종업원수_최소'].fillna(0)
df_meta['추출_종업원수_최대'] = df_meta['추출_종업원수_최대'].fillna(0)
for i in range(len(df_meta)):
    if df_meta['추출_종업원수_최소'][i] == '':
        df_meta['추출_종업원수_최소'][i]=0
    if df_meta['추출_종업원수_최대'][i] == '':
        df_meta['추출_종업원수_최대'][i] =0

# 추출 종업원수 최소값이 있는데 최대값은 없는 경우
for i in range(len(df_meta)):
    if (df_meta['추출_종업원수_최소'][i] != 0) and (df_meta['추출_종업원수_최대'][i] == 0):
        df_meta['추출_종업원수_최대'][i] = 9999999

### policy_df에 적용

In [ ]:
# policy_df['종업원수_최소'] = df_meta['추출_종업원수_최소']
# policy_df['종업원수_최대'] = df_meta['추출_종업원수_최대']

# 전체모델 run

In [ ]:

# 지역, 종업원의 경우 위 해당 셀 & 코드 실행 필수 (KoBert 사용)

policy_df['시도'] = df_meta['추출_시도']
policy_df['시군구'] = df_meta['추출_시군구']
policy_df['종업원수_최소'] = df_meta['추출_종업원수_최소']
policy_df['종업원수_최대'] = df_meta['추출_종업원수_최대']


# 주요 업종, 선호 특성, 설립 기간은 아래 코드로 실행 (Khaiii 사용)

def run_tag_model(df):

    df['context'] = df['서비스명'].astype(str) + df['소관기관'].astype(str)\
      + df['소관기관유형'].astype(str) + df['선정기준'].astype(str)\
      + df['서비스목적'].astype(str) + df['지원대상'].astype(str)

    api = KhaiiiApi()

    df['context_tokens'] = df['context'].apply(extract_nouns)
    df['주요 업종'] = df['context_tokens'].apply(big_category)
    df['선호 특성'] = df['context_tokens'].apply(characteristic)
    df['설립 기간'] = df['지원대상'].apply(founded_date)

    df['추출_설립기간_최소'] = df['추출_설립기간'].apply(foundate1)
    df['추출_설립기간_최대'] = df['추출_설립기간'].apply(foundate2)

    df.drop(['context', 'context_tokens', '추출_설립기간'], axis=1, inplace=True)

    print('태깅된 정책 데이터 :\n', df)

return df



In [ ]:
df = run_tag_model(policy_df)